# Build Action Corpus

We'll now build our corpus which stores actions that our bot can do. We
want to use Vectara as a recommendation engine for our Webhook when we
ask for a different task. We'll also use this to guide responses when the user
first starts chatting.

For this corpus we'll put in the following fields:
* **action-key:** This is our "key" which will trigger different actions within our Webhook
* **hidden:** Some actions should not be explicitly shown to a user, such as small talk
* **name:** This is the "human friendly" name for the action, which should be listed when the user first enters.

Each action will have a description of what it is to do - if the user enters a phrase which is similar, the system will ask the user whether they want to do "X" action.

## Initialize our Client
Once again, we initialize our client using implicit configuration.

We also initialize our logger and mute the OAuthUtil.

In [1]:
from vectara_client.core import Factory
from vectara_client.admin import CorpusBuilder
import logging

logging.basicConfig(format='%(asctime)s:%(name)-35s %(levelname)s:%(message)s', level=logging.INFO, datefmt='%H:%M:%S %z')
logging.getLogger("OAuthUtil").setLevel(logging.WARNING)
logger = logging.getLogger(__name__)

client = Factory().build()
manager = client.corpus_manager


13:23:15 +1000:Factory                             INFO:initializing builder
13:23:15 +1000:Factory                             INFO:Factory will load configuration from home directory
13:23:15 +1000:HomeConfigLoader                    INFO:Loading configuration from users home directory [C:\Users\david]
13:23:15 +1000:HomeConfigLoader                    INFO:Loading default configuration [default]
13:23:15 +1000:HomeConfigLoader                    INFO:Parsing config
13:23:15 +1000:root                                INFO:We are processing authentication type [OAuth2]
13:23:15 +1000:root                                INFO:initializing Client


## Create the Corpus
We'll again use our CorpusBuilder to create our corpus.

In [21]:
corpus = (CorpusBuilder("meetup-actions")
         .description("Provides appropriate actions for user content using recommendations")
         .add_attribute("action-key", "This is our \"key\" which will trigger different actions within our Webhook")
         .add_attribute("hidden", "Whether the action is hidden from the end user")
         .add_attribute("name","This is the \"human friendly\" name for the action, which should be listed when the user first enters.")
         .build())
corpus_id = manager.create_corpus(corpus, delete_existing=True)

14:15:50 +1000:CorpusManager                       INFO:Performing account checks before corpus creation for name [meetup-actions]
14:15:50 +1000:RequestUtil                         INFO:URL for operation list-corpora is: https://api.vectara.io/v1/list-corpora
14:15:51 +1000:CorpusManager                       INFO:Checking corpus with name [meetup-actions]
14:15:51 +1000:CorpusManager                       INFO:We found the following corpora with name [meetup-actions]: [727]
14:15:51 +1000:CorpusManager                       INFO:We found existing corpus with name [meetup-actions]
14:15:51 +1000:CorpusManager                       INFO:Deleting existing corpus named [meetup-actions]
14:15:51 +1000:RequestUtil                         INFO:URL for operation list-corpora is: https://api.vectara.io/v1/list-corpora
14:15:53 +1000:CorpusManager                       INFO:We found [1] potential matches
14:15:53 +1000:CorpusManager                       INFO:Deleting existing corpus with id [

## Load Corpus with our Actions

In [22]:
from pathlib import Path
import json

with open(Path("../resources/actions/bot-tasks.json"), "r") as f:
    for line in f.readlines():
        action = json.loads(line)
        logger.info(f"Found line:\n{json.dumps(action, indent=4)}")

        metadata = {
            "action-key": action["key"],
            "name": action["task"],
            "hidden": action["hidden"]
        }
        metadata_json = json.dumps(metadata)
        
        to_index = {
          "document_id": action["name"],
          "title": action["name"],
          "metadata_json": metadata_json,
          "section": [
            {
              "text": action["description"]
            }
          ]
        }

        client.indexer_service.index_doc(corpus_id, to_index)

14:16:03 +1000:__main__                            INFO:Found line:
{
    "task": "General Information",
    "key": "general",
    "description": "Provide general information about DataEngBytes",
    "hidden": "false"
}
14:16:03 +1000:RequestUtil                         INFO:URL for operation index is: https://api.vectara.io/v1/index
14:16:06 +1000:__main__                            INFO:Found line:
{
    "task": "Meetup Information",
    "key": "meetup-info",
    "description": "Provide information about meetups",
    "hidden": "false"
}
14:16:06 +1000:RequestUtil                         INFO:URL for operation index is: https://api.vectara.io/v1/index
14:16:09 +1000:__main__                            INFO:Found line:
{
    "task": "Submit Presentation",
    "key": "meetup-submission",
    "description": "Submit a talk or presentation for a meetup",
    "hidden": "false"
}
14:16:09 +1000:RequestUtil                         INFO:URL for operation index is: https://api.vectara.io/v1/in

## Test the Actions
Lets confirm the "top" actions for different user prompts and confirm our
recommendation engine is working as expected.

In [34]:
def check_action(query, expected_action=None, min_score=0):
    
    responses = client.query_service.query(query, corpus_id, semantics="RESPONSE",summary=False)

    response = responses.response[0]
    
    document_index = response.documentIndex
    document = responses.document[document_index]
    
    action = ""
    for metadata in document.metadata:
        if metadata.name == "action-key":
            action = metadata.value
    
    logger.info(f"For user text, [{query}], Score [{response.score}], Action [{action}]")
    if expected_action and action != expected_action:
        raise Exception(f"We were expecting action [{expected_action}] but received [{action}]")
    if response.score < min_score:
        raise Exception(f"The action found was too weakly matched [{response.score}] and did not meet our minimun score of [{min_score}]")

check_action("I would like to submit my talk")
check_action("I would like to submit my talk", "meetup-submission", .99)
check_action("Can we talk about DataEngBytes", "general", .99)
check_action("What else can you do?", "overview", .80)


14:36:47 +1000:RequestUtil                         INFO:URL for operation query is: https://api.vectara.io/v1/query
14:36:49 +1000:__main__                            INFO:For user text, [I would like to submit my talk], Score [0.9997863], Action [meetup-submission]
14:36:49 +1000:RequestUtil                         INFO:URL for operation query is: https://api.vectara.io/v1/query
14:36:50 +1000:__main__                            INFO:For user text, [I would like to submit my talk], Score [0.9997863], Action [meetup-submission]
14:36:50 +1000:RequestUtil                         INFO:URL for operation query is: https://api.vectara.io/v1/query
14:36:52 +1000:__main__                            INFO:For user text, [Can we talk about DataEngBytes], Score [1.0281286], Action [general]
14:36:52 +1000:RequestUtil                         INFO:URL for operation query is: https://api.vectara.io/v1/query
14:36:53 +1000:__main__                            INFO:For user text, [What else can you do?